In [1]:

question = "What is the tallest Pokemon?"
model_id = "chainyo/alpaca-lora-7b"
prompt_template = """Question: {question}

Answer: Let's think step by step."""



In [3]:
import requests
import shutil 
from tempfile import NamedTemporaryFile

def download_file(url: str) -> str:
    out = NamedTemporaryFile(mode="wb", delete=False)
    with requests.get(url, stream=True) as r:
        shutil.copyfileobj(r.raw, out)

    return out.name

ontology = download_file("https://www.pokemonkg.org/ontology/ontology.nt")
instances = download_file("https://www.pokemonkg.org/download/dump/poke-a.nq.gz")

In [2]:
import torch
from langchain.llms.base import LLM
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex, PromptHelper
from llama_index import LLMPredictor, ServiceContext
from transformers import pipeline
from typing import Optional, List, Mapping, Any


# define prompt helper
# set maximum input size
max_input_size = 2048
# set number of output tokens
num_output = 256
# set maximum chunk overlap
max_chunk_overlap = 20
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)


class CustomLLM(LLM):
    model_name = model_id
    pipeline = pipeline("text-generation", model=model_name, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt_length = len(prompt)
        response = self.pipeline(prompt, max_new_tokens=num_output)[0]["generated_text"]

        # only return newly generated tokens
        return response[prompt_length:]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "custom"

# define our LLM
llm_predictor = LLMPredictor(llm=CustomLLM())

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

# Load the your data
documents = SimpleDirectoryReader('./data').load_data()
index = GPTListIndex.from_documents(documents, service_context=service_context)

# Query and print response
response = index.query("<query_text>")
print(response)

/home/cyril/.cache/pypoetry/virtualenvs/aikg-URVQdnEY-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 39/39 [00:37<00:00,  1.05it/s]
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 0 tokens


None


In [8]:
from llama_index import download_loader, GPTSimpleVectorIndex
RDFReader = download_loader("RDFReader")
document = RDFReader().load_data(file=ontology)

Exception: Label not found for: http://www.w3.org/2002/07/owl#ObjectProperty

In [4]:

import re
from langchain import PromptTemplate

# Detect input {variables} from template
variables = re.findall(r"{(\w+)}", prompt_template)
prompt = PromptTemplate(
    template=prompt_template, input_variables=variables
)


In [10]:
from langchain import HuggingFacePipeline, LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

import dotenv
dotenv.load_dotenv()

True

In [11]:

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [12]:

model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 39/39 [00:38<00:00,  1.00it/s]


In [13]:

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128)

In [14]:

llm = HuggingFacePipeline(pipeline=pipe)

In [15]:

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [17]:

print(llm_chain.run("Give me 5 water pokemons?"))

 First, you need to define what a "water pokemon" is. Is it a pokemon that is part of the Water type? Is it a pokemon that has a water-related move? Is it a pokemon that has a water-related ability?

Once you have defined what a "water pokemon" is, you can then ask the question: "Give me 5 water pokemons?"

The answer to this question is:

1. Magikarp
2. Gyarados
3. Poliwag
4. Poliwrath
5. Squirt
